In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('statlog.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,1
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,1
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,1
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,1


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 0, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,0
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,0
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,0
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,0


In [5]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [7]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [8]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [9]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [10]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [11]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
3/3 [==============================] - 2s 248ms/step - loss: 0.8130 - accuracy: 0.5704 - val_loss: 0.6799 - val_accuracy: 0.5704
Epoch 2/100
3/3 [==============================] - 0s 60ms/step - loss: 0.5522 - accuracy: 0.7630 - val_loss: 0.6668 - val_accuracy: 0.6741
Epoch 3/100
3/3 [==============================] - 0s 64ms/step - loss: 0.4115 - accuracy: 0.7778 - val_loss: 0.6538 - val_accuracy: 0.6963
Epoch 4/100
3/3 [==============================] - 0s 60ms/step - loss: 0.4381 - accuracy: 0.8074 - val_loss: 0.6432 - val_accuracy: 0.7333
Epoch 5/100
3/3 [==============================] - 0s 60ms/step - loss: 0.3995 - accuracy: 0.8444 - val_loss: 0.6379 - val_accuracy: 0.7185
Epoch 6/100
3/3 [==============================] - 0s 64ms/step - loss: 0.3928 - accuracy: 0.8444 - val_loss: 0.6353 - val_accuracy: 0.7185
Epoch 7/100
3/3 [==============================] - 0s 30ms/step - loss: 0.4372 - accuracy: 0.8222 - val_loss: 0.6359 - val_accuracy: 0.7259
Epoch 8/100
3/3 [==

3/3 [==============================] - 0s 70ms/step - loss: 0.0486 - accuracy: 0.9852 - val_loss: 0.5377 - val_accuracy: 0.7333
Epoch 60/100
3/3 [==============================] - 0s 64ms/step - loss: 0.0438 - accuracy: 0.9926 - val_loss: 0.5328 - val_accuracy: 0.7407
Epoch 61/100
3/3 [==============================] - 0s 64ms/step - loss: 0.0539 - accuracy: 0.9778 - val_loss: 0.5316 - val_accuracy: 0.7259
Epoch 62/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0504 - accuracy: 0.9852 - val_loss: 0.5348 - val_accuracy: 0.7407
Epoch 63/100
3/3 [==============================] - 0s 26ms/step - loss: 0.0733 - accuracy: 0.9630 - val_loss: 0.5382 - val_accuracy: 0.7259
Epoch 64/100
3/3 [==============================] - 0s 34ms/step - loss: 0.0316 - accuracy: 0.9926 - val_loss: 0.5364 - val_accuracy: 0.7259
Epoch 65/100
3/3 [==============================] - 0s 30ms/step - loss: 0.0455 - accuracy: 0.9852 - val_loss: 0.5374 - val_accuracy: 0.7259
Epoch 66/100
3/3 [========

In [12]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


5/5 [==============================] - 0s 3ms/step - loss: 0.5316 - accuracy: 0.7259
Test Loss: 0.5316
Test Accuracy: 0.7259


In [13]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


5/5 [==============================] - 0s 3ms/step
Confusion Matrix:
[[66 14]
 [23 32]]

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.82      0.78        80
           1       0.70      0.58      0.63        55

    accuracy                           0.73       135
   macro avg       0.72      0.70      0.71       135
weighted avg       0.72      0.73      0.72       135



In [14]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [15]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [16]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
3/3 [==============================] - 13s 1s/step - loss: 0.6840 - accuracy: 0.7259 - val_loss: 0.6898 - val_accuracy: 0.7481
Epoch 2/100
3/3 [==============================] - 0s 117ms/step - loss: 0.6563 - accuracy: 0.7556 - val_loss: 0.6826 - val_accuracy: 0.6963
Epoch 3/100
3/3 [==============================] - 0s 120ms/step - loss: 0.6247 - accuracy: 0.7407 - val_loss: 0.6711 - val_accuracy: 0.6963
Epoch 4/100
3/3 [==============================] - 0s 116ms/step - loss: 0.5796 - accuracy: 0.7481 - val_loss: 0.6549 - val_accuracy: 0.6889
Epoch 5/100
3/3 [==============================] - 0s 126ms/step - loss: 0.5581 - accuracy: 0.7407 - val_loss: 0.6344 - val_accuracy: 0.6889
Epoch 6/100
3/3 [==============================] - 0s 126ms/step - loss: 0.5379 - accuracy: 0.7630 - val_loss: 0.6198 - val_accuracy: 0.6741
Epoch 7/100
3/3 [==============================] - 0s 70ms/step - loss: 0.5425 - accuracy: 0.7481 - val_loss: 0.6228 - val_accuracy: 0.6444
Epoch 8/100
3/3 

In [17]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


5/5 [==============================] - 0s 10ms/step - loss: 0.6198 - accuracy: 0.6741
Test Loss: 0.6198
Test Accuracy: 0.6741


In [18]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


5/5 [==============================] - 2s 10ms/step
Confusion Matrix:
[[77  3]
 [41 14]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.96      0.78        80
           1       0.82      0.25      0.39        55

    accuracy                           0.67       135
   macro avg       0.74      0.61      0.58       135
weighted avg       0.72      0.67      0.62       135



In [19]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [20]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [21]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
3/3 [==============================] - 19s 2s/step - loss: 0.6485 - accuracy: 0.6519 - val_loss: 0.6893 - val_accuracy: 0.6222
Epoch 2/100
3/3 [==============================] - 0s 191ms/step - loss: 0.5040 - accuracy: 0.7630 - val_loss: 0.6860 - val_accuracy: 0.6000
Epoch 3/100
3/3 [==============================] - 0s 190ms/step - loss: 0.4609 - accuracy: 0.8296 - val_loss: 0.6832 - val_accuracy: 0.6000
Epoch 4/100
3/3 [==============================] - 0s 186ms/step - loss: 0.4425 - accuracy: 0.8296 - val_loss: 0.6812 - val_accuracy: 0.5926
Epoch 5/100
3/3 [==============================] - 0s 196ms/step - loss: 0.3432 - accuracy: 0.8519 - val_loss: 0.6793 - val_accuracy: 0.5926
Epoch 6/100
3/3 [==============================] - 0s 195ms/step - loss: 0.3608 - accuracy: 0.8519 - val_loss: 0.6775 - val_accuracy: 0.5926
Epoch 7/100
3/3 [==============================] - 0s 195ms/step - loss: 0.3262 - accuracy: 0.8593 - val_loss: 0.6743 - val_accuracy: 0.5926
Epoch 8/100
3/3

In [22]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


5/5 [==============================] - 0s 18ms/step - loss: 0.6480 - accuracy: 0.6074
Test Loss: 0.6480
Test Accuracy: 0.6074


In [23]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


5/5 [==============================] - 3s 15ms/step
Confusion Matrix:
[[37 43]
 [10 45]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.46      0.58        80
           1       0.51      0.82      0.63        55

    accuracy                           0.61       135
   macro avg       0.65      0.64      0.61       135
weighted avg       0.67      0.61      0.60       135



In [24]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



5/5 [==============================] - 0s 16ms/step


In [25]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [26]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7185


In [27]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[66 14]
 [24 31]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.82      0.78        80
           1       0.69      0.56      0.62        55

    accuracy                           0.72       135
   macro avg       0.71      0.69      0.70       135
weighted avg       0.72      0.72      0.71       135

Ensemble Accuracy: 0.7185


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [29]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [30]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [31]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [32]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [33]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7852


In [34]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[69 11]
 [18 37]]
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.86      0.83        80
           1       0.77      0.67      0.72        55

    accuracy                           0.79       135
   macro avg       0.78      0.77      0.77       135
weighted avg       0.78      0.79      0.78       135

Ensemble Accuracy: 0.7852


In [35]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [36]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7556
Confusion Matrix:
[[68 12]
 [21 34]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.85      0.80        80
           1       0.74      0.62      0.67        55

    accuracy                           0.76       135
   macro avg       0.75      0.73      0.74       135
weighted avg       0.75      0.76      0.75       135

Ensemble Accuracy: 0.7556
